In [ ]:
!pip install datapackage
!pip install geopandas
!pip install folium

In [ ]:
import os, sys, subprocess
import numpy                 as np
import datapackage
import geopandas
import pandas                as pd
import matplotlib.pyplot     as plt
import seaborn               as sns
import folium  #needed for interactive map
from folium.plugins import HeatMap
%matplotlib inline
sns.set()

## Generate dataframe as csv with columns:
## 'Area', 'iso code', 'country area', 'cropland', 'cropland %'

In [ ]:
iso = pd.read_csv('iso.csv')
iso

,English short name,French short name,Alpha-2 code,Alpha-3 code,Numeric
0,Aruba,Aruba,AW,ABW,533
1,Afghanistan,Afghanistan (l'),AF,AFG,4
2,Angola,Angola (l'),AO,AGO,24
3,Anguilla,Anguilla,AI,AIA,660
4,Åland Islands,Åland(les Îles),AX,ALA,248
...,...,...,...,...,...
244,Samoa,Samoa (le),WS,WSM,882
245,Yemen,Yémen (le),YE,YEM,887
246,South Africa,Afrique du Sud (l'),ZA,ZAF,710
247,Zambia,Zambie (la),ZM,ZMB,894


In [ ]:
iso = iso.drop(columns=['French short name','Alpha-2 code','Numeric'])
iso = iso.rename(columns={"English short name" : "Area", "Alpha-3 code" : "iso"})
iso

,Area,iso
0,Aruba,ABW
1,Afghanistan,AFG
2,Angola,AGO
3,Anguilla,AIA
4,Åland Islands,ALA
...,...,...
244,Samoa,WSM
245,Yemen,YEM
246,South Africa,ZAF
247,Zambia,ZMB


In [ ]:
# Load landuse data
landuse = pd.read_csv('FAOSTAT_data_9-27-2021.csv')
landuse.head(3)

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,RL,Land Use,2,Afghanistan,5110,Area,6600,Country area,1961,1961,1000 ha,65286.0,W,Data reported on country official publications...
1,RL,Land Use,2,Afghanistan,5110,Area,6600,Country area,1962,1962,1000 ha,65286.0,W,Data reported on country official publications...
2,RL,Land Use,2,Afghanistan,5110,Area,6600,Country area,1963,1963,1000 ha,65286.0,W,Data reported on country official publications...


In [ ]:
# Remove any NaN Value rows 
landuse.dropna(subset=['Value'])
landuse.head()

# Remove unecessary columns
landuse = landuse.drop(columns=['Domain Code', 'Domain', 'Area Code (FAO)', 'Element Code','Element','Item Code','Year Code','Unit','Flag','Flag Description'])

# Filter all rows for which Value is in the follwing:
droplabels = ['Agriculture area actually irrigated', 'Cropland area actually irrigated', 'Farm buildings & farmyards']
for i in droplabels:
    landuse.drop(landuse[landuse['Item'] == i].index, inplace = True)

landuse

,Area,Item,Year,Value
0,Afghanistan,Country area,1961,65286.0
1,Afghanistan,Country area,1962,65286.0
2,Afghanistan,Country area,1963,65286.0
3,Afghanistan,Country area,1964,65286.0
4,Afghanistan,Country area,1965,65286.0
...,...,...,...,...
64639,Zimbabwe,Arable land,2015,4000.0
64640,Zimbabwe,Arable land,2016,4000.0
64641,Zimbabwe,Arable land,2017,4000.0
64642,Zimbabwe,Arable land,2018,4000.0


In [ ]:
# Find the 'Area' values in 'land' that don't match ISO values so I can fix them now

# Turn both into lists, and then take the non-intersection
matched = list(set(landuse['Area'].unique()).intersection(iso['Area'].unique()))

unmatched = list(set(landuse['Area'].unique()).difference(iso['Area'].unique()))

# This is how many countries were successfully matched
print('# of matched countries = ' + str(len(matched)))
print('# of unmatched countries = ' + str(len(unmatched)))

# Which countries are in the landuse dataset but were not matched
landuse_unmatched = list(set(landuse['Area'].unique()).difference(matched))
landuse_unmatched = sorted(landuse_unmatched)
print(landuse_unmatched)
print()
# Which countries are in the iso dataset but were not matched -- These are the only ones I can fix!!
iso_unmatched = list(set(iso['Area'].unique()).difference(matched))
iso_unmatched = sorted(iso_unmatched)
print(iso_unmatched)

# of matched countries = 193
# of unmatched countries = 54
['Bahamas', 'Belgium-Luxembourg', 'British Virgin Islands', "C?te d'Ivoire", 'Cayman Islands', 'Central African Republic', 'Channel Islands', 'China, Hong Kong SAR', 'China, Macao SAR', 'China, Taiwan Province of', 'China, mainland', 'Comoros', 'Congo', 'Cook Islands', 'Cura?ao', 'Czechoslovakia', "Democratic People's Republic of Korea", 'Democratic Republic of the Congo', 'Dominican Republic', 'Ethiopia PDR', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'French Guyana', 'Gambia', 'Holy See', "Lao People's Democratic Republic", 'Marshall Islands', 'Netherlands', 'Netherlands Antilles (former)', 'Niger', 'Northern Mariana Islands', 'Pacific Islands Trust Territory', 'Palestine', 'Philippines', 'R?union', 'Republic of Korea', 'Republic of Moldova', 'Russian Federation', 'Saint Barth?lemy', 'Saint-Martin (French part)', 'Serbia and Montenegro', 'Sudan', 'Sudan (former)', 'Syrian Arab Republic', 'Turks and Caicos Islands', 'USSR

In [ ]:
# Next step, manually replace the landuse['Area'] values so that they match the world['name'] values
# Then, delete all others from the landuse chart
landuse['Area'].replace('Bahamas','Bahamas (the)',inplace=True)
landuse['Area'].replace('Cayman Islands','Cayman Islands (the)',inplace=True)
landuse['Area'].replace('Central African Republic','Central African Republic (the)',inplace=True)
landuse['Area'].replace('British Virgin Islands','British Indian Ocean Territory (the)',inplace=True)
landuse['Area'].replace('Comoros','Comoros (the)',inplace=True)
landuse['Area'].replace('Congo','Congo (the)',inplace=True)
landuse['Area'].replace('Cook Islands','Cook Islands (the)',inplace=True)
landuse['Area'].replace('Cura?ao','Curaçao',inplace=True)
landuse['Area'].replace("C?te d'Ivoire","Côte d'Ivoire",inplace=True)
landuse['Area'].replace('Democratic Republic of the Congo','Congo (the Democratic Republic of the)',inplace=True)
landuse['Area'].replace('Falkland Islands (Malvinas)', 'Falkland Islands (the) [Malvinas]',inplace=True)
landuse['Area'].replace('Faroe Islands', 'Faroe Islands (the)',inplace=True)
landuse['Area'].replace('French Guyana', 'French Guiana',inplace=True)

landuse['Area'].replace("Democratic People's Republic of Korea", "Korea (the Democratic People's Republic of)",inplace=True)
landuse['Area'].replace('Republic of Korea', 'Korea (the Republic of)',inplace=True)
landuse['Area'].replace("Lao People's Democratic Republic", "Lao People's Democratic Republic (the)",inplace=True)

landuse['Area'].replace('Netherlands', 'Netherlands (the)',inplace=True)
landuse['Area'].replace('Niger', 'Niger (the)',inplace=True)
landuse['Area'].replace('Palestine', 'Palestine, State of',inplace=True)
landuse['Area'].replace('Philippines', 'Philippines (the)',inplace=True)
landuse['Area'].replace('Republic of Moldova', 'Moldova (the Republic of)',inplace=True)

landuse['Area'].replace('Gambia','Gambia (the)',inplace=True)
landuse['Area'].replace('Holy See', 'Holy See (the)',inplace=True)
landuse['Area'].replace('Marshall Islands', 'Marshall Islands (the)',inplace=True)
landuse['Area'].replace('Sudan', 'Sudan (the)',inplace=True)
landuse['Area'].replace('Western Sahara', 'Western Sahara*',inplace=True)

landuse['Area'].replace('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom of Great Britain and Northern Ireland (the)',inplace=True)
landuse['Area'].replace('United States Virgin Islands', 'Virgin Islands (U.S.)',inplace=True)
landuse['Area'].replace('United States of America','United States of America (the)',inplace=True)

landuse['Area'].replace('R?union', 'Réunion',inplace=True)
landuse['Area'].replace('United Republic of Tanzania', 'Tanzania, the United Republic of',inplace=True)
landuse['Area'].replace('Wallis and Futuna Islands','Wallis and Futuna',inplace=True)
landuse['Area'].replace('Syrian Arab Republic','Syrian Arab Republic (the)',inplace=True)

landuse['Area'].replace('Turks and Caicos Islands', 'Turks and Caicos Islands (the)',inplace=True)

landuse['Area'].replace('United Arab Emirates', 'United Arab Emirates (the)',inplace=True)
landuse['Area'].replace('Northern Mariana Islands', 'Northern Mariana Islands (the)',inplace=True)
landuse['Area'].replace('Russian Federation', 'Russian Federation (the)',inplace=True)
landuse['Area'].replace('United Arab Emirates', 'United Arab Emirates (the)',inplace=True)

landuse['Area'].replace('Saint Barth?lemy','Saint Barthélemy',inplace=True)
landuse['Area'].replace('Saint-Martin (French part)','Saint Martin (French part)',inplace=True)

landuse['Area'].replace('Dominican Republic', 'Dominican Republic (the)',inplace=True)


In [ ]:
# China is 're-used' 
# Here are all of the landuse 'areas' that have china = ['China', 'China, Hong Kong SAR', 'China, Macao SAR','China, mainland', 'Taiwan']

# Remove 'China'
droplabels = ['China']
for i in droplabels:
    landuse.drop(landuse[landuse['Area'] == i].index, inplace = True)

# rename Hong Kong, Macao, China mainland, and Taiwan
landuse['Area'].replace('China, Hong Kong SAR', 'Hong Kong',inplace=True)
landuse['Area'].replace('China, Macao SAR', 'Macao',inplace=True)
landuse['Area'].replace('China, mainland', 'China',inplace=True)
landuse['Area'].replace('China, Taiwan Province of', 'Taiwan (Province of China)',inplace=True)

In [ ]:
# Ethiopia and Ethiopia PDR are the same -- they have unique sets of years. Rename both to 'Ethiopia'.
landuse['Area'].replace('Ethiopia PDR', 'Ethiopia',inplace=True)

In [ ]:
# For the second time, find the 'Area' values in 'land' that don't match ISO values so I can fix them now

# Turn both into lists, and then take the non-intersection
matched = list(set(landuse['Area'].unique()).intersection(iso['Area'].unique()))

unmatched = list(set(landuse['Area'].unique()).difference(iso['Area'].unique()))

# This is how many countries were successfully matched
print('# of matched countries = ' + str(len(matched)))
print('# of unmatched countries = ' + str(len(unmatched)))

# Which countries are in the landuse dataset but were not matched
landuse_unmatched = list(set(landuse['Area'].unique()).difference(matched))
landuse_unmatched = sorted(landuse_unmatched)
print(landuse_unmatched)
print()
# Which countries are in the iso dataset but were not matched -- These are the only ones I can fix!!
iso_unmatched = list(set(iso['Area'].unique()).difference(matched))
iso_unmatched = sorted(iso_unmatched)
print(iso_unmatched)

# of matched countries = 236
# of unmatched countries = 9
['Belgium-Luxembourg', 'Channel Islands', 'Czechoslovakia', 'Netherlands Antilles (former)', 'Pacific Islands Trust Territory', 'Serbia and Montenegro', 'Sudan (former)', 'USSR', 'Yugoslav SFR']

['Antarctica', 'Bouvet Island', 'Christmas Island', 'Cocos (Keeling) Islands (the)', 'French Southern Territories (the)', 'Guernsey', 'Heard Island and McDonald Islands', 'Jersey', 'South Georgia and the South Sandwich Islands', 'Svalbard and Jan Mayen', 'United States Minor Outlying Islands (the)', 'Virgin Islands (British)', 'Åland Islands']


In [ ]:
# Print years for Sudan and Sudan (former)
print(country_area[country_area['Area'] == 'South Sudan'   ]['Year'].unique())
print(country_area[country_area['Area'] == 'Sudan (the)'   ]['Year'].unique())
print(country_area[country_area['Area'] == 'Sudan (former)']['Year'].unique())

[2012 2013 2014 2015 2016 2017 2018 2019]
[2012 2013 2014 2015 2016 2017 2018 2019 1961 1962 1963 1964 1965 1966
 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980
 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994
 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008
 2009 2010 2011]
[]


In [ ]:
# Print country_area for 'Sudan (the)', 'South Sudan', and 'Sudan (former)'
print(country_area[country_area['Area'] == 'South Sudan'])
print(country_area[country_area['Area'] == 'Sudan (the)'])
print(country_area[country_area['Area'] == 'Sudan (former)'])

              Area  Year  Country_Area
54202  South Sudan  2012       64688.3
54203  South Sudan  2013       64688.3
54204  South Sudan  2014       64688.3
54205  South Sudan  2015       64688.3
54206  South Sudan  2016       64688.3
54207  South Sudan  2017       64688.3
54208  South Sudan  2018       64688.3
54209  South Sudan  2019       64688.3
              Area  Year  Country_Area
54843  Sudan (the)  2012   185410.6027
54844  Sudan (the)  2013   185410.5341
54845  Sudan (the)  2014   185410.4772
54846  Sudan (the)  2015   185410.5787
54847  Sudan (the)  2016   185410.4915
54848  Sudan (the)  2017   185410.6923
54849  Sudan (the)  2018   185410.5405
54850  Sudan (the)  2019   185410.5405
54883  Sudan (the)  1961   250581.0000
54884  Sudan (the)  1962   250581.0000
54885  Sudan (the)  1963   250581.0000
54886  Sudan (the)  1964   250581.0000
54887  Sudan (the)  1965   250581.0000
54888  Sudan (the)  1966   250581.0000
54889  Sudan (the)  1967   250581.0000
54890  Sudan (the)  1968 

In [ ]:
# Area of Sudan (the) varies. Pick a year (2012) and sum up 'Sudan (the)' and 'South Sudan'. Compare to 'Sudan (former)'
a = sum(country_area[(country_area['Area'] == 'South Sudan') & (country_area['Year'] == 2012)]['Country_Area'],
        country_area[(country_area['Area'] == 'Sudan (the)') & (country_area['Year'] == 2012)]['Country_Area']
       )
b = country_area[(country_area['Area'] == 'Sudan (former)') & (country_area['Year'] == 1990)]['Country_Area']

print(a)
print(b)

54843    250098.9027
Name: Country_Area, dtype: float64
Series([], Name: Country_Area, dtype: float64)


In [ ]:
# According to the ISO, when Sudan split into Sudan and South Sudan, they did NOT change the Alpha-3 code for Sudan (former) and Sudan.
# They just added an additional entry to the country list for 'South Sudan' and left 'Sudan' unchanged.
# We will go similarly, and rename 'Sudan (former)' as 'Sudan'
landuse['Area'].replace('Sudan (former)', 'Sudan (the)',inplace=True)

In [ ]:
# Add these 9 unmatched countries to iso

new_iso = ['Czechoslovakia', 'CSK',
            'Netherlands Antilles (former)', 'ANT',
            'Pacific Islands Trust Territory', 'PCI',
            'Serbia and Montenegro', 'SCG',
            'USSR','SUN',
            'Yugoslav SFR','YUG',
            'Belgium-Luxembourg','BLX'
            ]
new_iso = np.reshape(new_iso,(7,2))

In [ ]:
# Make sure none of these ISO codes aren't already included in the iso list
for i in new_iso[:,1]:
    print(iso[iso['iso'] == i].sum())

Area    0.0
iso     0.0
dtype: float64
Area    0.0
iso     0.0
dtype: float64
Area    0.0
iso     0.0
dtype: float64
Area    0.0
iso     0.0
dtype: float64
Area    0.0
iso     0.0
dtype: float64
Area    0.0
iso     0.0
dtype: float64
Area    0.0
iso     0.0
dtype: float64


In [ ]:
# Add new_iso to the iso dataframe
new_iso = pd.DataFrame(new_iso, columns=('Area','iso'))
iso = iso.append(new_iso)

In [ ]:
# Check again, how many unmatched countries?
# Turn both into lists, and then take the non-intersection
matched = list(set(landuse['Area'].unique()).intersection(iso['Area'].unique()))

unmatched = list(set(landuse['Area'].unique()).difference(iso['Area'].unique()))

# This is how many countries were successfully matched
print('# of matched countries = ' + str(len(matched)))
print('# of unmatched countries = ' + str(len(unmatched)))

# Which countries are in the landuse dataset but were not matched
landuse_unmatched = list(set(landuse['Area'].unique()).difference(matched))
landuse_unmatched = sorted(landuse_unmatched)
print(landuse_unmatched)
print()
# Which countries are in the iso dataset but were not matched -- These are the only ones I can fix!!
iso_unmatched = list(set(iso['Area'].unique()).difference(matched))
iso_unmatched = sorted(iso_unmatched)
print(iso_unmatched)

# of matched countries = 243
# of unmatched countries = 1
['Channel Islands']

['Antarctica', 'Bouvet Island', 'Christmas Island', 'Cocos (Keeling) Islands (the)', 'French Southern Territories (the)', 'Guernsey', 'Heard Island and McDonald Islands', 'Jersey', 'South Georgia and the South Sandwich Islands', 'Svalbard and Jan Mayen', 'United States Minor Outlying Islands (the)', 'Virgin Islands (British)', 'Åland Islands']


In [ ]:
landuse['Item'].unique()

array(['Country area', 'Land area', 'Agricultural land', 'Cropland',
       'Arable land'], dtype=object)

In [ ]:
# Extract each of these into their own pandas dataset
country_area = landuse[landuse['Item'] == 'Country area']
cropland = landuse[landuse['Item'] == 'Cropland']

# Rename 'Value' column names and drop 'Item' column
cropland = cropland.rename(columns={"Value": "Cropland"})
cropland = cropland.drop(columns=['Item'], axis=1)

country_area = country_area.rename(columns={"Value": "Country_Area"})
country_area = country_area.drop(columns=['Item'], axis=1)

In [ ]:
# shape of cropland
np.shape(cropland)

(12352, 3)

In [ ]:
# shape of country_area
np.shape(country_area)

(12917, 3)

In [ ]:
# Merge datasets into 'land'
land = cropland.merge(country_area)
land

,Area,Year,Cropland,Country_Area
0,Afghanistan,1961,7700.0,65286.0
1,Afghanistan,1962,7760.0,65286.0
2,Afghanistan,1963,7810.0,65286.0
3,Afghanistan,1964,7873.0,65286.0
4,Afghanistan,1965,7875.0,65286.0
...,...,...,...,...
12347,Zimbabwe,2015,4100.0,39076.0
12348,Zimbabwe,2016,4100.0,39076.0
12349,Zimbabwe,2017,4100.0,39076.0
12350,Zimbabwe,2018,4100.0,39076.0


In [ ]:
# Add percentage cropland to a new column
land['Cropland_pct'] = land['Cropland']/land['Country_Area']
land

,Area,Year,Cropland,Country_Area,Cropland_pct
0,Afghanistan,1961,7700.0,65286.0,0.117943
1,Afghanistan,1962,7760.0,65286.0,0.118862
2,Afghanistan,1963,7810.0,65286.0,0.119627
3,Afghanistan,1964,7873.0,65286.0,0.120592
4,Afghanistan,1965,7875.0,65286.0,0.120623
...,...,...,...,...,...
12347,Zimbabwe,2015,4100.0,39076.0,0.104924
12348,Zimbabwe,2016,4100.0,39076.0,0.104924
12349,Zimbabwe,2017,4100.0,39076.0,0.104924
12350,Zimbabwe,2018,4100.0,39076.0,0.104924


In [ ]:
# Merge iso into land
land = land.merge(iso)
land

,Area,Year,Cropland,Country_Area,Cropland_pct,iso
0,Afghanistan,1961,7700.0,65286.0,0.117943,AFG
1,Afghanistan,1962,7760.0,65286.0,0.118862,AFG
2,Afghanistan,1963,7810.0,65286.0,0.119627,AFG
3,Afghanistan,1964,7873.0,65286.0,0.120592,AFG
4,Afghanistan,1965,7875.0,65286.0,0.120623,AFG
...,...,...,...,...,...,...
12288,Zimbabwe,2015,4100.0,39076.0,0.104924,ZWE
12289,Zimbabwe,2016,4100.0,39076.0,0.104924,ZWE
12290,Zimbabwe,2017,4100.0,39076.0,0.104924,ZWE
12291,Zimbabwe,2018,4100.0,39076.0,0.104924,ZWE


In [ ]:
# are any empty on the iso column?
land['iso'].isnull().values.any()

False

In [ ]:
land.to_csv('land.csv')

## Generate EDA plots of cropland data

In [ ]:
land.head()

NameError: ignored

In [ ]:
# Sum data across all countries, separated by Item code, for each year

# Start by creating a table with just Year, Item, and Value Information
global_data = land.copy(deep=True)
a = global_data.groupby(["Year"]).Cropland.sum()
b = global_data.groupby(["Year"]).Country_Area.sum()
c = global_data.groupby(["Year"]).Cropland_pct.mean()
d = global_data.groupby(["Year"]).Cropland_pct.sum()
global_data = pd.DataFrame([a,b,c,d])
temp = global_data.T.copy(deep=True)
temp.columns = ['Cropland','Country_Area', 'Mean_Cropland_pct','Sum_Cropland_pct']
global_data = temp.copy(deep=True)

global_data.head(3)

In [ ]:
# Load the geopandas world dataset
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world.head(10)

In [ ]:
world.plot();

In [ ]:
# Plot just arable land over time
# Let's plot each of them on the same plot
fig, axs = plt.subplots()
sns.lineplot(x='Year', y='Cropland', data=land)
fig.suptitle('Global Cropland in thousand hectares')
plt.tight_layout()
fig.show()

In [ ]:
# Plotting Cropland versus Country Area in 2019
fig, ax = plt.subplots()
sns.scatterplot(ax=ax, y='Cropland',x='Country_Area',data=land[land['Year']==2019])
ax.set_ylabel('Cropland (thousand hectares)')
ax.set_xlabel('Country Area (thousand hectares)')
fig.suptitle('Cropland versus Country Area in 2019')

In [ ]:
# Plotting Cropland % versus Country Area in 2019

fig, ax = plt.subplots()

sns.scatterplot(ax=ax, y='Cropland_pct',x='Country_Area', data=land[land['Year']==2019])
ax.set_ylabel('Sum of Cropland %')
ax.set_ylabel('Cropland %')
ax.set_xlabel('Country Area (thousand hectares)')
fig.suptitle('Cropland % versus Country Area in 2019')

In [ ]:
fig, ax = plt.subplots(1,2)

sns.lineplot(ax=ax[0], x='Year',y='Mean_Cropland_pct',data=global_data)
ax[0].set_ylabel('Mean of Cropland %')

sns.lineplot(ax=ax[1], x='Year',y='Sum_Cropland_pct',data=global_data)
ax[1].set_ylabel('Sum of Cropland %')

fig.suptitle('Cropland % over time')
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(ax = ax, x='Year', y='Cropland_pct', data=land)
ax.set_ylabel('Percentage')
fig.suptitle('Percentage of Country Area designated as Cropland')

In [ ]:
sns.lineplot(x='Year', y='Cropland_pct', hue='iso', data=land)
#place legend outside top right corner of plot
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
fig = sns.lineplot(x='Year', y='Cropland_pct', hue='Area', data=land)
# remove legend
fig.legend_.remove()

In [ ]:
decades = np.linspace(1960,2020,7)
decades = [int(x) for x in decades]
print(decades)

ticks = np.linspace(0,len(land['Year'].unique()),7)

fig, ax = plt.subplots()
sns.boxplot(ax=ax, x='Year',y='Cropland_pct', data=land)
ax.set_xticks(ticks)
ax.set_xticklabels(decades)
ax.set_ylabel('Percentage')
fig.suptitle('Percentage of Country Area designated as Cropland')
